In [4]:
mlist = [
#"bert-base-uncased",
##"microsoft/deberta-large-mnli",
#"xlm-roberta-large",
"microsoft/deberta-base-mnli"
]

import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import mlflow
import torch


#df = pd.read_csv('drive_download/classification_data_subset.csv')
df = pd.read_csv("drive_download/final_class.csv")
#df.kategorie_id = df.kategorie_id - 1
#df = df.rename(columns={"kategorie_id":"label", "inhalt":"text"})
#df = df.drop(columns=['kategorie_bezeichnung', 'titel', 'inhalt_kurz'])
df = df[df['text'].notna()]
traindf, testdf = train_test_split(df, test_size=0.2)

train = Dataset.from_pandas(traindf)
test = Dataset.from_pandas(testdf)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test
device = "cuda" if torch.cuda.is_available() else "cpu"
for x in mlist:
  
  print(x)
  
  from transformers import AutoTokenizer
  model_name = x

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  

  def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)

  tokenized_datasets = dataset.map(preprocess_function, batched=True)
  
  from transformers import DataCollatorWithPadding

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  
  from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

  model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                            num_labels=7,
                                                            ignore_mismatched_sizes=True).to(device)
  
  import numpy as np
  from datasets import load_metric
  metric = load_metric("accuracy")
  #%%
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
  
  num_epochs = 2

  args = TrainingArguments(
      output_dir ='./results',          
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 16,   
      per_device_eval_batch_size = 16,   
      weight_decay = 0.01,               
      learning_rate = 2e-5,             
      metric_for_best_model = 'accuracy',    
      evaluation_strategy = "epoch",
      )


  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets['train'],
      eval_dataset=tokenized_datasets['test'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )  
  mlflow.autolog()
  trainer.train()
  mlflow.log_param("model",x)
  mlflow.end_run()

microsoft/deberta-base-mnli


loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--microsoft--deberta-base-mnli/snapshots/a80a6eb013898011540b19bf1f64e21eb61e53d6/config.json
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base-mnli",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "CONTRADICTION",
    "1": "NEUTRAL",
    "2": "ENTAILMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "CONTRADICTION": 0,
    "ENTAILMENT": 2,
    "NEUTRAL": 1
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "c2p",
    "p2c"

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--microsoft--deberta-base-mnli/snapshots/a80a6eb013898011540b19bf1f64e21eb61e53d6/config.json
Model config DebertaConfig {
  "_name_or_path": "microsoft/deberta-base-mnli",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_tok

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.016183,0.996032
2,0.097700,0.012855,0.996984


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6300
  Batch size = 64
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.

In [15]:
#trainer.save_model("classmodel")
#tokenizer.save_pretrained("classmodel")

tokenizer config file saved in classmodel/tokenizer_config.json
Special tokens file saved in classmodel/special_tokens_map.json


('classmodel/tokenizer_config.json',
 'classmodel/special_tokens_map.json',
 'classmodel/vocab.json',
 'classmodel/merges.txt',
 'classmodel/added_tokens.json',
 'classmodel/tokenizer.json')